In [1]:
!pip install datasets

In [2]:
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from moviepy.editor import VideoFileClip
from keras import layers, applications
from datasets import load_dataset
from tensorflow import keras
from tqdm import tqdm
import torchvision.transforms.functional as F
import torch.nn.functional as FF
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2, os, gc, torch, time

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [6]:
while True :
    try :
        ds = load_dataset('nexar-ai/nexar_collision_prediction', split='train')
        break

    except :
        time.sleep(40)

Resolving data files:   0%|          | 0/1502 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1348 [00:00<?, ?it/s]

In [7]:
n = round(ds.num_rows * 0.6)
subset = ds.shuffle(seed=1404).select(range(n))

In [8]:
train_df = subset.to_pandas()

In [9]:
train_df['label'] = train_df['time_of_event'].apply(lambda x: 0 if pd.isna(x) or x is None else 1)

In [10]:
train_df = pd.get_dummies(
    train_df,
    columns=['light_conditions', 'weather', 'scene'],
    prefix=['light', 'weather', 'scene'],
    dtype=int
)

In [11]:
train_df_split, val_df_split = train_test_split(train_df, test_size=0.3, random_state=1404)

In [12]:
output_dir = 'video_matrices'
os.makedirs(output_dir, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def resize_on_gpu(frames):
    # Convert (N, H, W, C) -> (N, C, H, W)
    frames_tensor = torch.from_numpy(frames).permute(0, 3, 1, 2).float().to(device)
    # Resize to (224, 224)
    resized = F.resize(frames_tensor, [224, 224])
    # Back to (N, H, W, C)
    return resized.permute(0, 2, 3, 1).cpu().numpy()

def video_to_matrix(video_df, is_train=True, frame_interval=6):
    for idx, row in tqdm(video_df.iterrows(), total=len(video_df)):
        video_path = row['video']['path']
        time_of_event = row['time_of_event']
        label = row['label']

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f'[WARN] Cannot open video at index {idx}: {video_path}')
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0 or fps is None:
            print(f'[WARN] FPS is zero at index {idx}')
            cap.release()
            continue

        total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        duration = total_frames / fps

        if label :
            start_time = time_of_event - 2
            end_time = time_of_event
        else :
            random_time = np.random.uniform(0, duration - 2)
            start_time = random_time
            end_time = random_time + 2

        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        frames = []
        frame_index = start_frame

        while frame_index <= end_frame:
            success, frame = cap.read()
            if not success:
                break
            if frame_index % frame_interval == 0:
                frames.append(frame)
            frame_index += 1

        cap.release()

        if frames:
            frames = np.stack(frames)  # (N, H, W, 3)
            video_matrix = resize_on_gpu(frames)  # GPU accelerated resize
            save_path = os.path.join(output_dir, f'video_{idx}.npy')
            np.save(save_path, video_matrix)

        del frames
        gc.collect()


# Run it
video_to_matrix(train_df_split)
video_to_matrix(val_df_split, is_train=False)

Using device: cuda


100%|██████████| 270/270 [06:30<00:00,  1.45s/it]


In [13]:
def add_matrix_path_col(df) :
  output_dir = 'video_matrices'
  df['video_matrix_path'] = None

  for idx in df.index:
      file_path = os.path.join(output_dir, f'video_{idx}.npy')
      if os.path.exists(file_path):
          df.at[idx, 'video_matrix_path'] = file_path
      else:
          df.at[idx, 'video_matrix_path'] = None

add_matrix_path_col(train_df_split)
add_matrix_path_col(val_df_split)

In [14]:
def masked_video_to_matrix(df):
    output_dir = 'masked_video_matrices'
    os.makedirs(output_dir, exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    for idx, video in tqdm(df.iterrows(), total=len(df)):
        video_path = '/content/' + video['video_matrix_path']
        video_frames = np.load(video_path)  # (N, H, W, 3)
        frames_tensor = torch.from_numpy(video_frames).permute(0, 3, 1, 2).float().to(device) / 255.0
        num_frames = frames_tensor.shape[0]

        mask_frames = []

        # Convert to grayscale on GPU
        gray = 0.2989 * frames_tensor[:, 0] + 0.5870 * frames_tensor[:, 1] + 0.1140 * frames_tensor[:, 2]

        for i in range(1, num_frames):
            prev_gray = gray[i - 1:i]  # (1, H, W)
            next_gray = gray[i:i + 1]

            # Optical flow approximation using spatial gradients
            # Compute gradients with Sobel kernels
            sobel_x = torch.tensor([[1, 0, -1],
                                    [2, 0, -2],
                                    [1, 0, -1]], dtype=torch.float32, device=device).view(1, 1, 3, 3)
            sobel_y = torch.tensor([[1, 2, 1],
                                    [0, 0, 0],
                                    [-1, -2, -1]], dtype=torch.float32, device=device).view(1, 1, 3, 3)

            Ix = FF.conv2d(prev_gray.unsqueeze(0), sobel_x, padding=1)
            Iy = FF.conv2d(prev_gray.unsqueeze(0), sobel_y, padding=1)
            It = next_gray.unsqueeze(0) - prev_gray.unsqueeze(0)

            # Lucas–Kanade flow per-pixel in 3×3 window
            kernel = torch.ones((1, 1, 3, 3), device=device)
            Ixx = FF.conv2d(Ix * Ix, kernel, padding=1)
            Iyy = FF.conv2d(Iy * Iy, kernel, padding=1)
            Ixy = FF.conv2d(Ix * Iy, kernel, padding=1)
            Ixt = FF.conv2d(Ix * It, kernel, padding=1)
            Iyt = FF.conv2d(Iy * It, kernel, padding=1)

            det = Ixx * Iyy - Ixy * Ixy + 1e-6
            u = (Iyy * (-Ixt) - Ixy * (-Iyt)) / det
            v = (-Ixy * (-Ixt) + Ixx * (-Iyt)) / det

            u, v = u[0, 0], v[0, 0]  # (H, W)

            # Magnitude / angle
            magnitude = torch.sqrt(u ** 2 + v ** 2)
            angle = torch.atan2(v, u) * (180.0 / np.pi / 2.0)

            # Normalize magnitude → [0,255]
            magnitude = 255 * (magnitude - magnitude.min()) / (magnitude.max() - magnitude.min() + 1e-6)

            # Divergence
            dy_u, dx_u = torch.gradient(u)
            dy_v, dx_v = torch.gradient(v)
            divergence = dx_u + dy_v
            divergence = torch.tanh(divergence)
            divergence = ((divergence + 1.0) / 2.0) * 255

            flow_channels = torch.stack([magnitude, angle, divergence], dim=0).clamp(0, 255)
            mask_frames.append(flow_channels.to("cpu", torch.uint8))

        mask_frames = torch.stack(mask_frames).numpy()
        save_path = os.path.join(output_dir, f'masked_video_{idx}.npy')
        np.save(save_path, mask_frames)

        del frames_tensor, mask_frames
        gc.collect()

masked_video_to_matrix(train_df_split)
masked_video_to_matrix(val_df_split)

Using device: cuda


100%|██████████| 630/630 [06:53<00:00,  1.52it/s]


Using device: cuda


100%|██████████| 270/270 [02:56<00:00,  1.53it/s]


In [15]:
def add_matrix_path_col(df) :
  output_dir = 'masked_video_matrices'
  df['masked_video_matrix_path'] = None

  for idx in df.index:
      file_path = os.path.join(output_dir, f'masked_video_{idx}.npy')
      if os.path.exists(file_path) :
          df.at[idx, 'masked_video_matrix_path'] = file_path
      else:
          df.at[idx, 'masked_video_matrix_path'] = None

add_matrix_path_col(train_df_split)
add_matrix_path_col(val_df_split)

In [16]:
train_df_split = train_df_split.reset_index()
val_df_split = val_df_split.reset_index()

In [17]:
train_df_split = train_df_split.drop(columns=['video', 'time_of_event', 'time_of_alert', 'time_to_accident', 'index'])
val_df_split = val_df_split.drop(columns=['video', 'time_of_event', 'time_of_alert', 'time_to_accident', 'index'])

In [18]:
MAX_FRAMES = 10
def pad_or_truncate(seq, max_len):
    seq = np.array(seq)
    if len(seq) > max_len:
        return seq[:max_len]
    if len(seq) < max_len:
        last = seq[-1]
        padding = np.repeat(last[None, ...], max_len - len(seq), axis=0)
        return np.concatenate([seq, padding], axis=0)
    return seq

class VideoMatrixSequence(keras.utils.Sequence):
    def __init__(self, df, batch_size=16, target_size=(224, 224), shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.columns = self.df.columns.drop(['video_matrix_path', 'masked_video_matrix_path', 'label'])
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return len(self.df) // self.batch_size

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]

        tabular_data = []
        video_data = []
        mask_data = []
        labels = []

        for i in batch_indices :
            row = self.df.iloc[i]

            video = np.load(row['video_matrix_path'])
            frames_resized = np.stack(video)
            frames_resized = pad_or_truncate(frames_resized, MAX_FRAMES)

            mask = np.load(row['masked_video_matrix_path'])
            mask_frames = np.stack(mask)
            mask_frames = pad_or_truncate(mask_frames, MAX_FRAMES)

            tabular_features = [row[col] for col in self.columns]
            tabular_data.append(tabular_features)

            video_data.append(frames_resized)
            mask_data.append(mask_frames)
            labels.append(float(row['label']))

        tabular_array = np.array(tabular_data, dtype=np.float32)
        video_array = np.array(video_data, dtype=np.float32)
        mask_array = np.array(mask_data, dtype=np.float32)
        labels_array = np.array(labels, dtype=np.float32)
        labels_array = np.expand_dims(labels_array, axis=-1)

        output = (
            {
                'video': video_array,
                'mask_flow': mask_array,
                'tabular': tabular_array
            }, labels_array
        )
        return output

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [19]:
train_seq = VideoMatrixSequence(train_df_split, batch_size=16, target_size=(224, 224))
val_seq = VideoMatrixSequence(val_df_split, batch_size=16, target_size=(224, 224))

In [20]:
def augmentation_pipeline(noise_std=0.1, brightness_factor=0.2, contrast_factor=0.2):
    return tf.keras.Sequential([
        layers.RandomBrightness(factor=brightness_factor),
        layers.RandomContrast(factor=contrast_factor),
        layers.GaussianNoise(noise_std)
    ], name="augmentation_pipeline")

In [21]:
def mask_cnn_net(input_shape=(3, 224, 224), dropout_rate=0.5):
    inputs = layers.Input(shape=input_shape)
    x = layers.Permute((2, 3, 1))(inputs)
    x = keras.layers.Normalization(mean=0., variance=255.)(x)
    # x = layers.TimeDistributed(augmentation_pipeline())(x)

    base = applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_tensor=x,
        pooling=None
    )
    base.trainable = False

    x = base.output
    x = layers.GlobalAveragePooling2D()(x)

    return keras.Model(inputs, x, name='mask_flow_backbone')

In [22]:
IMG_SIZE = 224
NUM_TABULAR_FEATURES = 14

# Inputs
video_input = keras.Input(shape=(MAX_FRAMES, IMG_SIZE, IMG_SIZE, 3), name='video')
mask_input = keras.Input(shape=(MAX_FRAMES, 3, IMG_SIZE, IMG_SIZE), name='mask_flow')
tabular_input = keras.Input(shape=(NUM_TABULAR_FEATURES,), name='tabular')

# Pre trained weights
def preprocess_resnet(x):
    return applications.resnet50.preprocess_input(x)

video_x = layers.TimeDistributed(layers.Lambda(preprocess_resnet))(video_input)
# video_x = layers.TimeDistributed(augmentation_pipeline())(video_x)

# CNN Backbone
base_cnn = applications.ResNet50(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_cnn.trainable = False

video_features = layers.TimeDistributed(base_cnn)(video_x)
mask_features = layers.TimeDistributed(mask_cnn_net())(mask_input)

# GRU
video_gru_out = layers.Bidirectional(layers.GRU(128, return_sequences=False))(video_features)
video_gru_out = layers.Dropout(0.3)(video_gru_out)

mask_gru_out = layers.Bidirectional(layers.GRU(128, return_sequences=False))(mask_features)
mask_gru_out = layers.Dropout(0.3)(mask_gru_out)

# Merge video and mask
video_final = layers.Concatenate()([video_gru_out, mask_gru_out])

# Tabular part
tabular_dense = layers.Dense(64, activation='relu')(tabular_input)
tabular_final = layers.Dropout(0.3)(tabular_dense)

# Merge all
final_merge = layers.Concatenate()([video_final, tabular_final])

final_merge = layers.BatchNormalization()(final_merge)
final_merge = layers.Dropout(0.3)(final_merge)

outputs = layers.Dense(1, activation='sigmoid')(final_merge)

# Final model
model = keras.Model(
    inputs=[video_input, mask_input, tabular_input],
    outputs=outputs,
    name='final_model'
)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.AUC(name='auc'),
        keras.metrics.FBetaScore(beta=1.0, threshold=0.5, average='macro', name='f1'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

cbs = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='best_video_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(train_seq, validation_data=val_seq, epochs=40, callbacks=cbs)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


  self._warn_if_super_not_called()



Epoch 1/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5398 - auc: 0.5513 - f1: 0.5440 - loss: 0.9227 - precision: 0.5408 - recall: 0.5482
Epoch 1: val_accuracy improved from -inf to 0.72266, saving model to best_video_model.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.5413 - auc: 0.5538 - f1: 0.5455 - loss: 0.9192 - precision: 0.5423 - recall: 0.5499 - val_accuracy: 0.7227 - val_auc: 0.8184 - val_f1: 0.7455 - val_loss: 0.5619 - val_precision: 0.6624 - val_recall: 0.8525
Epoch 2/40
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7375 - auc: 0.8176 - f1: 0.7418 - loss: 0.5469 - precision: 0.7335 - recall: 0.7508
Epoch 2: val_accuracy improved from 0.72266 to 0.75781, saving model to best_video_model.keras
39/39 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.7380 - auc: 0.8177 - f1: 0.7423 - loss: 0.5468 - precision: 0.7339 - recall: 0.7512 - val_accuracy: 0.7578 - val_auc: 0.8409 - val_f1: 0.7615 - val_loss: 0.5135 - val_precision: 0.6972 - val_recall: 0.8390
